## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Domino Data Lab Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. You need to make sure that these datasets are uploaded and present in the same directory as this notebook file, since we will be importing these files in Spark and create the following tables under the __`country_club`__ database using Spark SQL.

1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into Spark to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the Domino platform, but for the final deliverable, please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

# Checking Existence of Spark Environment Variables

Make sure your notebook is loaded using a PySpark Workspace. If you open up a regular Jupyter workspace the following variables might not exist

In [3]:
spark

Out[ 3 ]: <pyspark.sql.session.SparkSession at 0x7f20ba3d47f0>

In [4]:
sqlContext

Out[ 4 ]: <pyspark.sql.context.HiveContext at 0x7f20ba3d4f60>

### Run the following if you failed to open a notebook in the PySpark Workspace

This will work assuming you are using Spark in the cloud on domino or you might need to configure with your own spark instance if you are working offline

In [6]:
if 'sc' not in locals():
    from pyspark.context import SparkContext
    from pyspark.sql.context import SQLContext
    from pyspark.sql.session import SparkSession
    
    sc = SparkContext()
    sqlContext = SQLContext(sc)
    spark = SparkSession(sc)

# Create a utility function to run SQL commands

Instead of typing the same python functions repeatedly, we build a small function where you can just pass your query to get results.

- Remember we are using Spark SQL in PySpark
- We can't run multiple SQL statements in one go (no semi-colon ';' separated SQL statements)
- We can run multi-line SQL queries (but still has to be a single statement)

In [8]:
def run_sql(statement):
    try:
        result = sqlContext.sql(statement)
    except Exception as e:
        print(e.desc, '\n', e.stackTrace)
        return
    return result

# Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [10]:
run_sql('drop database if exists country_club cascade')
run_sql('create database country_club')
dbs = run_sql('show databases')
dbs.toPandas()

Out[ 7 ]: 
 databaseName
0 country_club
1 default

# Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you.

To get started, first make sure you have already uploaded the three CSV files and they are present in the same directory as the notebook.

Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [12]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "True"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [14]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
-- bookid: string (nullable = true)
-- facid: string (nullable = true)
-- memid: string (nullable = true)
-- starttime: string (nullable = true)
-- slots: string (nullable = true)

Facilities Schema
root
-- facid: string (nullable = true)
-- name: string (nullable = true)
-- membercost: string (nullable = true)
-- guestcost: string (nullable = true)
-- initialoutlay: string (nullable = true)
-- monthlymaintenance: string (nullable = true)

Members Schema
root
-- memid: string (nullable = true)
-- surname: string (nullable = true)
-- firstname: string (nullable = true)
-- address: string (nullable = true)
-- zipcode: string (nullable = true)
-- telephone: string (nullable = true)
-- recommendedby: string (nullable = true)
-- joindate: string (nullable = true)

# Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [17]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [19]:
run_sql('use country_club')
run_sql('REFRESH table bookings')
run_sql('REFRESH table facilities')
run_sql('REFRESH table members')
tbls = run_sql('show tables')
tbls.toPandas()

Out[ 11 ]: 
 database tableName isTemporary
0 country_club bookings False
1 country_club facilities False
2 country_club members False

# Test a sample SQL query

__Note:__ You can use multi-line SQL queries (but still a single statement) as follows

In [21]:
result = run_sql('''
                    SELECT * 
                    FROM bookings 
                    LIMIT 3
                 ''')
result.toPandas()

Out[ 12 ]: 
 bookid facid memid starttime slots
0 0 3 1 2012-07-03 11:00:00 2
1 1 4 1 2012-07-03 08:00:00 2
2 2 6 0 2012-07-03 18:00:00 2

# Your Turn: Solve the following questions with Spark SQL

- Make use of the `run_sql(...)` function as seen in the previous example
- You can write multi-line SQL queries but it has to be a single statement (no use of semi-colons ';')
- Make use of the `toPandas()` function as depicted in the previous example to display the query results

In [23]:
facilities_df.head(1000)

Out[ 13 ]: 
[Row(facid='0', name='Tennis Court 1', membercost='5.0', guestcost='25.0', initialoutlay='10000', monthlymaintenance='200'),
 Row(facid='1', name='Tennis Court 2', membercost='5.0', guestcost='25.0', initialoutlay='8000', monthlymaintenance='200'),
 Row(facid='2', name='Badminton Court', membercost='0.0', guestcost='15.5', initialoutlay='4000', monthlymaintenance='50'),
 Row(facid='3', name='Table Tennis', membercost='0.0', guestcost='5.0', initialoutlay='320', monthlymaintenance='10'),
 Row(facid='4', name='Massage Room 1', membercost='9.9', guestcost='80.0', initialoutlay='4000', monthlymaintenance='3000'),
 Row(facid='5', name='Massage Room 2', membercost='9.9', guestcost='80.0', initialoutlay='4000', monthlymaintenance='3000'),
 Row(facid='6', name='Squash Court', membercost='3.5', guestcost='17.5', initialoutlay='5000', monthlymaintenance='80'),
 Row(facid='7', name='Snooker Table', membercost='0.0', guestcost='5.0', initialoutlay='450', monthlymaintenance='15'),
 Row(facid='8', name='Pool Table', membercost='0.0', guestcost='5.0', initialoutlay='400', monthlymaintenance='15')]

#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [25]:
result = run_sql('''
                  Select facid, name from facilities
                  where membercost >0
                   
                 ''')
result.toPandas()

Out[ 14 ]: 
 facid name
0 0 Tennis Court 1
1 1 Tennis Court 2
2 4 Massage Room 1
3 5 Massage Room 2
4 6 Squash Court

####  Q2: How many facilities do not charge a fee to members?

In [27]:
result = run_sql('''
                 Select facid, name from facilities
                  where membercost ==0
                 ''')
result.toPandas()

Out[ 15 ]: 
 facid name
0 2 Badminton Court
1 3 Table Tennis
2 7 Snooker Table
3 8 Pool Table

#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [29]:
result = run_sql('''
                 Select facid, name, membercost, monthlymaintenance from facilities
                 where membercost < 0.2*monthlymaintenance and membercost>0
                ''')
result.toPandas()

Out[ 16 ]: 
 facid name membercost monthlymaintenance
0 0 Tennis Court 1 5.0 200
1 1 Tennis Court 2 5.0 200
2 4 Massage Room 1 9.9 3000
3 5 Massage Room 2 9.9 3000
4 6 Squash Court 3.5 80

#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [31]:
result = run_sql('''
                select * from facilities
                where facid==1
                Union all
                select * from facilities
                where facid==5
                   
                ''')
result.toPandas()

Out[ 17 ]: 
 facid name membercost guestcost initialoutlay monthlymaintenance
0 1 Tennis Court 2 5.0 25.0 8000 200
1 5 Massage Room 2 9.9 80.0 4000 3000

#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [33]:
result = run_sql('''
                 select monthlymaintenance,
                 case when monthlymaintenance > 100 then "expensive"
                 ELSE "cheap" END AS name
                 from facilities
                 ''')
result.toPandas()

Out[ 18 ]: 
 monthlymaintenance name
0 200 expensive
1 200 expensive
2 50 cheap
3 10 cheap
4 3000 expensive
5 3000 expensive
6 80 cheap
7 15 cheap
8 15 cheap

In [34]:
members_df.head(5)

Out[ 19 ]: 
[Row(memid='0', surname='GUEST', firstname='GUEST', address='GUEST', zipcode='0', telephone='(000) 000-0000', recommendedby=None, joindate='2012-07-01 00:00:00'),
 Row(memid='1', surname='Smith', firstname='Darren', address='8 Bloomsbury Close, Boston', zipcode='4321', telephone='555-555-5555', recommendedby=None, joindate='2012-07-02 12:02:05'),
 Row(memid='2', surname='Smith', firstname='Tracy', address='8 Bloomsbury Close, New York', zipcode='4321', telephone='555-555-5555', recommendedby=None, joindate='2012-07-02 12:08:23'),
 Row(memid='3', surname='Rownam', firstname='Tim', address='23 Highway Way, Boston', zipcode='23423', telephone='(844) 693-0723', recommendedby=None, joindate='2012-07-03 09:32:15'),
 Row(memid='4', surname='Joplette', firstname='Janice', address='20 Crossing Road, New York', zipcode='234', telephone='(833) 942-4710', recommendedby='1', joindate='2012-07-03 10:25:05')]

#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [36]:
result = run_sql('''
                 SELECT surname, firstname FROM members
                 where joindate == "2012-09-26 18:08:45"
                 order by joindate desc
                 ''')
result.toPandas()

Out[ 20 ]: 
 surname firstname
0 Smith Darren

In [37]:
bookings_df.head(5)

Out[ 21 ]: 
[Row(bookid='0', facid='3', memid='1', starttime='2012-07-03 11:00:00', slots='2'),
 Row(bookid='1', facid='4', memid='1', starttime='2012-07-03 08:00:00', slots='2'),
 Row(bookid='2', facid='6', memid='0', starttime='2012-07-03 18:00:00', slots='2'),
 Row(bookid='3', facid='7', memid='1', starttime='2012-07-03 19:00:00', slots='2'),
 Row(bookid='4', facid='8', memid='1', starttime='2012-07-03 10:00:00', slots='1')]

In [38]:
facilities_df.head(5)

Out[ 22 ]: 
[Row(facid='0', name='Tennis Court 1', membercost='5.0', guestcost='25.0', initialoutlay='10000', monthlymaintenance='200'),
 Row(facid='1', name='Tennis Court 2', membercost='5.0', guestcost='25.0', initialoutlay='8000', monthlymaintenance='200'),
 Row(facid='2', name='Badminton Court', membercost='0.0', guestcost='15.5', initialoutlay='4000', monthlymaintenance='50'),
 Row(facid='3', name='Table Tennis', membercost='0.0', guestcost='5.0', initialoutlay='320', monthlymaintenance='10'),
 Row(facid='4', name='Massage Room 1', membercost='9.9', guestcost='80.0', initialoutlay='4000', monthlymaintenance='3000')]

In [39]:
members_df.head(5)

Out[ 23 ]: 
[Row(memid='0', surname='GUEST', firstname='GUEST', address='GUEST', zipcode='0', telephone='(000) 000-0000', recommendedby=None, joindate='2012-07-01 00:00:00'),
 Row(memid='1', surname='Smith', firstname='Darren', address='8 Bloomsbury Close, Boston', zipcode='4321', telephone='555-555-5555', recommendedby=None, joindate='2012-07-02 12:02:05'),
 Row(memid='2', surname='Smith', firstname='Tracy', address='8 Bloomsbury Close, New York', zipcode='4321', telephone='555-555-5555', recommendedby=None, joindate='2012-07-02 12:08:23'),
 Row(memid='3', surname='Rownam', firstname='Tim', address='23 Highway Way, Boston', zipcode='23423', telephone='(844) 693-0723', recommendedby=None, joindate='2012-07-03 09:32:15'),
 Row(memid='4', surname='Joplette', firstname='Janice', address='20 Crossing Road, New York', zipcode='234', telephone='(833) 942-4710', recommendedby='1', joindate='2012-07-03 10:25:05')]

####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [41]:
result = run_sql('''select distinct concat (com1.surname, " ", com1.firstname) as person_name, fac.name from (
                 select book.facid, book.memid, mem.surname, mem.firstname from bookings as booK
                 join members as mem on book.memid = mem.memid
                 Where mem.surname <> "GUEST"
                 ) as com1
                 join facilities as fac on com1.facid =fac.facid
                 where fac.name like "Tennis%"
                 order by person_name
                 ''')
result.toPandas()

Out[ 24 ]: 
 person_name name
0 Bader Florence Tennis Court 1
1 Bader Florence Tennis Court 2
2 Baker Anne Tennis Court 2
3 Baker Anne Tennis Court 1
4 Baker Timothy Tennis Court 1
5 Baker Timothy Tennis Court 2
6 Boothe Tim Tennis Court 1
7 Boothe Tim Tennis Court 2
8 Butters Gerald Tennis Court 1
9 Butters Gerald Tennis Court 2
10 Coplin Joan Tennis Court 1
11 Crumpet Erica Tennis Court 1
12 Dare Nancy Tennis Court 2
13 Dare Nancy Tennis Court 1
14 Farrell David Tennis Court 2
15 Farrell David Tennis Court 1
16 Farrell Jemima Tennis Court 2
17 Farrell Jemima Tennis Court 1
18 Genting Matthew Tennis Court 1
19 Hunt John Tennis Court 1
20 Hunt John Tennis Court 2
21 Jones David Tennis Court 2
22 Jones David Tennis Court 1
23 Jones Douglas Tennis Court 1
24 Joplette Janice Tennis Court 2
25 Joplette Janice Tennis Court 1
26 Owen Charles Tennis Court 2
27 Owen Charles Tennis Court 1
28 Pinker David Tennis Court 1
29 Purview Millicent Tennis Court 2
30 Rownam Tim Tennis Court 1
31 Rownam Tim Tennis Court 2
32 Rumney Henrietta Tennis Court 2
33 Sarwin Ramnaresh Tennis Court 1
34 Sarwin Ramnaresh Tennis Court 2
35 Smith Darren Tennis Court 2
36 Smith Jack Tennis Court 2
37 Smith Jack Tennis Court 1
38 Smith Tracy Tennis Court 1
39 Smith Tracy Tennis Court 2
40 Stibbons Ponder Tennis Court 2
41 Stibbons Ponder Tennis Court 1
42 Tracy Burton Tennis Court 1
43 Tracy Burton Tennis Court 2

#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [43]:
result = run_sql('''
                    select concat (surname, " ", firstname) as personal_name, f.name as facility_name, 2*f.membercost*b.slots as cost from bookings as b inner join 
                    members as m on m.memid = b.memid inner join facilities as f on f.facid=b.facid
                    where m.memid<>0 and 2*f.membercost*b.slots >30 and b.starttime between "2012-09-14 00:00:00" and "2012-09-14 24:00:00"
                    Union all
                    select concat (surname, " ", firstname) as personal_name, f.name as facility_name, f.guestcost*b.slots as cost from bookings as b inner join 
                    members as m on m.memid = b.memid inner join facilities as f on f.facid=b.facid
                    where m.memid=0 and f.guestcost*b.slots >30 and b.starttime between "2012-09-14 00:00:00" and "2012-09-14 24:00:00"
                    order by cost
       
                 ''')
result.toPandas()

Out[ 204 ]: 
 personal_name facility_name cost
0 GUEST GUEST Squash Court 35.0
1 GUEST GUEST Squash Court 35.0
2 Smith Jack Massage Room 1 39.6
3 Farrell Jemima Massage Room 1 39.6
4 Stibbons Ponder Massage Room 1 39.6
5 Tracy Burton Massage Room 1 39.6
6 Genting Matthew Massage Room 1 39.6
7 Bader Florence Massage Room 2 39.6
8 Jones David Tennis Court 2 60.0
9 Boothe Tim Tennis Court 2 60.0
10 GUEST GUEST Squash Court 70.0
11 GUEST GUEST Tennis Court 1 75.0
12 GUEST GUEST Tennis Court 1 75.0
13 GUEST GUEST Tennis Court 2 75.0
14 Farrell Jemima Massage Room 1 79.2
15 GUEST GUEST Tennis Court 2 150.0
16 GUEST GUEST Massage Room 1 160.0
17 GUEST GUEST Massage Room 1 160.0
18 GUEST GUEST Massage Room 1 160.0
19 GUEST GUEST Massage Room 2 320.0

#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [45]:
result = run_sql('''
                   select personal_name, starttime,facility_name, cost from (select personal_name, starttime, facility_name, memid, guecost, case when memid=0 then guecost else memcost end as cost from 
                   (select personal_name, memid, starttime, f.name as facility_name, f.guestcost*tem.slots as guecost, f.membercost*2*tem.slots as memcost 
                   from (select concat (surname, " ", firstname) as personal_name, b.memid as memid, facid, b.starttime as starttime, slots 
                   from bookings as b inner join members as m on m.memid = b.memid) as tem 
                   inner join facilities f on f.facid=tem.facid) as mid_tem)
                   where cost >30 and memid=0 and starttime between "2012-09-14 00:00:00" and "2012-09-14 24:00:00" or memid<>0 and cost >30 and starttime between "2012-09-14 00:00:00" and "2012-09-14 24:00:00"
                   order by cost
                   
                   
                   
                   
                   
                   
   
                 ''')
result.toPandas()

Out[ 205 ]: 
 personal_name starttime facility_name cost
0 GUEST GUEST 2012-09-14 12:30:00 Squash Court 35.0
1 GUEST GUEST 2012-09-14 15:00:00 Squash Court 35.0
2 Smith Jack 2012-09-14 08:00:00 Massage Room 1 39.6
3 Farrell Jemima 2012-09-14 11:00:00 Massage Room 1 39.6
4 Stibbons Ponder 2012-09-14 12:00:00 Massage Room 1 39.6
5 Tracy Burton 2012-09-14 18:00:00 Massage Room 1 39.6
6 Genting Matthew 2012-09-14 19:00:00 Massage Room 1 39.6
7 Bader Florence 2012-09-14 09:30:00 Massage Room 2 39.6
8 Jones David 2012-09-14 08:00:00 Tennis Court 2 60.0
9 Boothe Tim 2012-09-14 11:00:00 Tennis Court 2 60.0
10 GUEST GUEST 2012-09-14 09:30:00 Squash Court 70.0
11 GUEST GUEST 2012-09-14 16:00:00 Tennis Court 1 75.0
12 GUEST GUEST 2012-09-14 19:00:00 Tennis Court 1 75.0
13 GUEST GUEST 2012-09-14 14:00:00 Tennis Court 2 75.0
14 Farrell Jemima 2012-09-14 14:00:00 Massage Room 1 79.2
15 GUEST GUEST 2012-09-14 17:00:00 Tennis Court 2 150.0
16 GUEST GUEST 2012-09-14 09:00:00 Massage Room 1 160.0
17 GUEST GUEST 2012-09-14 13:00:00 Massage Room 1 160.0
18 GUEST GUEST 2012-09-14 16:00:00 Massage Room 1 160.0
19 GUEST GUEST 2012-09-14 11:00:00 Massage Room 2 320.0

#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [47]:
result = run_sql('''
                 select facid, sum(cost) as final_cost from
                 (select concat (surname, " ", firstname) as personal_name, b.memid as memid, b.facid as facid, f.name as name, f.membercost as memcost, f.guestcost as guecost,b.starttime as starttime, slots, case when b.memid=0 then f.guestcost*slots else f.membercost*2*slots end as cost
                   from bookings as b inner join members as m on m.memid = b.memid
                   inner join facilities f on f.facid= b.facid)
                   group by facid
                   having final_cost<1000
                 ''')
result.toPandas()

Out[ 228 ]: 
 facid final_cost
0 7 240.0
1 3 180.0
2 8 270.0